# 1. Package preparation

In [6]:
#! pip install missingno
#! pip install geopy
# !pip install geopandas
# !pip install geoplot   # need conda install -c conda-forge cartopy 
# !pip install shapely
import numpy as np
import pandas as pd
import geopy
np.set_printoptions(threshold=np.inf)
from scipy import stats
import seaborn as sns
%matplotlib inline
from matplotlib.pylab import plt
%pylab inline
import missingno as msno 
import warnings # ignore the warnings
# from geopandas import GeoDataFrame
# import geopandas as gpd
# from shapely.geometry import Point, Polygon
# from geopy.geocoders import Nominatim
# import geoplot as gplt
from string import ascii_letters
import seaborn as sns
import scipy.stats as ss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [7]:
UShousing2 = pd.read_csv('UShousing2.csv')
UShousing2.shape

(6383, 12)

In [8]:
UShousing3 = UShousing2.copy()
X = UShousing3.drop('rent_amount',axis=1)
y = UShousing2['rent_amount']
X.shape, y.shape

((6383, 11), (6383,))

# 2. Feature Engineering

a. Convert categorical features to numerics
b. Standardize Numerical features

In [9]:
UShousing2 = UShousing2[UShousing2['rent_amount'] > 0]
xreg = pd.get_dummies(data=X[X.columns.difference(["identity"])])

# 3. Decision Tree Model Implement

##### Split Training set & Testing set

In [10]:
X_train, X_test, y_train, y_test = train_test_split(xreg, y, test_size=0.3, random_state= 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4468, 2357), (1915, 2357), (4468,), (1915,))

In [11]:
X_train.shape

(4468, 2357)

In [12]:
numerics = ['int64','float64']

train_num = X_train.select_dtypes(include=numerics)
test_num =  X_test[train_num.columns]


scaler = StandardScaler()
x_train_scaled = pd.DataFrame(scaler.fit_transform(train_num),columns = train_num.columns)
x_test_scaled = pd.DataFrame(scaler.transform(test_num), columns = test_num.columns)

x_train_scaled.shape,x_test_scaled.shape
x_train_scaled.head()

,bathroom_count,bedroom_count,rent_duration,year
0,-0.638320,0.057880,-0.013735,-1.306077
1,0.701282,0.772273,-0.013735,1.258437
2,-0.638320,-0.656512,-0.013735,1.258437
3,-0.638320,-0.656512,-0.013735,-1.306077
4,1.371084,0.057880,-0.013735,-0.023820


In [13]:
X_train.loc[:,'bathroom_count'] = x_train_scaled.loc[:,'bathroom_count'].values
X_train.loc[:,'bedroom_count'] = x_train_scaled.loc[:,'bedroom_count'].values
X_train.loc[:,'rent_duration'] = x_train_scaled.loc[:,'rent_duration'].values
X_test.loc[:,'bathroom_count'] = x_test_scaled.loc[:,'bathroom_count'].values
X_test.loc[:,'bedroom_count'] = x_test_scaled.loc[:,'bedroom_count'].values
X_test.loc[:,'rent_duration'] = x_test_scaled.loc[:,'rent_duration'].values

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
import sklearn.metrics as metrics
grid_values = {'ccp_alpha': np.linspace(0, 0.1, 21), 'min_samples_split':[2,10,50,100]}
dtc = DecisionTreeRegressor(random_state=88)
dtc_cv = GridSearchCV(dtc, param_grid=grid_values, cv=5, n_jobs=-1).fit(X_train, y_train)
y_pred = dtc_cv.predict(X_test)
print('MAE: ',  metrics.mean_absolute_error(y_test, y_pred))
print('MSE: ',  metrics.mean_squared_error(y_test, y_pred))

MAE:  622.7612884703385
MSE:  1305667.5168203241
